In [67]:
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import ADASYN, SMOTE, BorderlineSMOTE
from sklearn.preprocessing import StandardScaler  # doctest: +SKIP
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
import numpy as np
from sklearn.utils import class_weight
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, Model
from sklearn.preprocessing import LabelBinarizer
#from sklearn.metrics.roc_auc_score
from sklearn import svm
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import warnings

warnings.filterwarnings("ignore")


scaler = StandardScaler()  

X = pd.read_csv('features.csv',encoding='windows-1252',sep=',')

X.fillna(value=0, inplace=True)

X.guten_genre = pd.factorize(X.guten_genre)[0]
X['index1'] = X.index

#y = X.guten_genre#.get_dummies()
y = X.guten_genre #dumiies

#lb = LabelBinarizer()
#y = lb.fit_transform(X.guten_genre)

print(y)

X = X.drop('guten_genre', 1)

X.head()

0      0
1      1
2      1
3      2
4      1
      ..
991    1
992    1
993    0
994    0
995    1
Name: guten_genre, Length: 996, dtype: int64


,p_tag_count,comma_count,colon_count,hyphen_count,TTR,Positive,Neutral,Negative,FleschK_score,CC,PRP,PRP$,IN,NNP,period_count,index1
0,2283,5669,18,850,0.154,0.194,0.673,0.133,51.398,22.0,6.0,0,688.0,17.0,382883,0
1,268,962,54,311,0.348,0.232,0.651,0.118,41.849,8.0,2.0,0,137.0,4.0,104178,1
2,3510,11771,415,2664,0.113,0.240,0.618,0.142,58.161,23.0,15.0,8,1473.0,23.0,821156,2
3,3332,5798,17,1358,0.137,0.164,0.729,0.107,64.087,56.0,29.0,4,1011.0,23.0,506644,3
4,472,1429,39,282,0.262,0.254,0.587,0.159,54.611,4.0,1.0,0,237.0,1.0,98120,4


In [71]:
modeller = TopicModeller()
modeller.setNumberOfTopics(4)
modeller.setPercentage(0.1)
modeller.loadData( 'prepared_tokens.npy' )

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM",
         "Decision Tree", "Random Forest", "Neural Net", 
         "Naive Bayes", "QDA"]

#classifiers = [
   #GaussianNB(),
   #KNeighborsClassifier(5),
   #DecisionTreeClassifier(max_depth=5),
  # MLPClassifier(solver='lbfgs', alpha=0.000001, random_state=None)
 #                   #hidden_layer_sizes=(5, 2), 
#]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

sm = SMOTE(random_state = None, k_neighbors=1)

#skf = StratifiedKFold(n_splits=3)

#ros = RandomOverSampler(random_state=None)
#rus = RandomUnderSampler(random_state=None)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

df_train, df_test = modeller.getFeatures(X_train.index1, X_test.index1)    

X_train = X_train.join(df_train)
X_test = X_test.join(df_test)
X_train.fillna(value=0, inplace=True)
X_test.fillna(value=0, inplace=True)


X_train = X_train.drop('index1',axis=1)
X_test = X_test.drop('index1',axis=1)
#X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
#X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
#X_resampled, y_resampled = ADASYN(n_neighbors=1).fit_resample(X_train, y_train)


scaler.fit(X_resampled) 
    
X_resampled = scaler.transform(X_resampled)

X_test = scaler.transform(X_test)

for name, clf in zip(names,classifiers):
  clf.fit(X_resampled, y_resampled)
  score = clf.score(X_test, y_test)
  y_pred = clf.predict(X_test)
  print(str(name) + ", Accuracy: " + str(score) + ", F1-Score: " + str(f1_score(y_test,y_pred, average = 'weighted')))
 

  #y_test = lb.inverse_transform(y_test, threshold=None)
  #y_pred = lb.inverse_transform(y_pred)
  print(confusion_matrix(y_test, y_pred))

  print('###########')


Nearest Neighbors, Accuracy: 0.505, F1-Score: 0.5746080586080586
[[ 7 11  0  0  0  0  4  0  0]
 [30 94  2  1  3  8 19  1  2]
 [ 0  1  0  0  0  1  2  0  0]
 [ 0  0  0  0  1  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0]
 [ 5  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0]]
###########
Linear SVM, Accuracy: 0.465, F1-Score: 0.5492428320160283
[[10  6  2  0  0  1  3  0  0]
 [19 79  6  9  2 11 24  1  9]
 [ 0  3  0  0  0  0  1  0  0]
 [ 0  1  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0]
 [ 0  1  0  0  1  2  0  0  0]
 [ 2  2  0  1  0  0  2  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0]]
###########
RBF SVM, Accuracy: 0.8, F1-Score: 0.7111111111111111
[[  0  22   0   0   0   0   0   0]
 [  0 160   0   0   0   0   0   0]
 [  0   4   0   0   0   0   0   0]
 [  0   1   0   0   0   0   0   0]
 [  0   1   0   0   0   0   0   0]
 [  0   4   0   0   0   0   0   0]
 [  0   7   0   0   0   0   0   0]
 [  0   1   0

In [54]:
from sklearn.model_selection import StratifiedShuffleSplit
import gensim
from gensim import corpora
import numpy as np
import random

class TopicModeller():

  def __init__( self ):
    self.genres = []
    self.books = []
    self.NUM_TOPICS = 10
    self.PERCENTAGE = 0.2

  def setNumberOfTopics( self, value ):
    self.NUM_TOPICS = value

  def setPercentage( self, value ):
    self.PERCENTAGE = value

  def loadData( self, path ):
    with open(path, 'rb') as f:
        self.genres = np.load(f, allow_pickle=True)
        self.books = np.load(f, allow_pickle=True)

  def getFeatures( self, trainIndices, testIndices ):

    books_train, books_test = self.books[trainIndices], self.books[testIndices]
    genres_train, genres_test = self.genres[trainIndices], self.genres[testIndices]

    arr = []

    for i in range( len(genres_train ) ):
      if genres_train[i] == 'Literary':
        val = random.random()
        if val > self.PERCENTAGE:
          continue
      arr.append( books_train[i])

    # Create the topics
    NUM_WORDS  = 4

    dictionary = corpora.Dictionary( arr )
    dictionary.filter_extremes(no_below=20, no_above=0.5)
    corpus = [ dictionary.doc2bow( text ) for text in arr ]

    # Set training parameters.
    num_topics = self.NUM_TOPICS
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = None  # Don't evaluate model perplexity, takes too much time.

    # Make a index to word dictionary.
    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token

    ldamodel = gensim.models.ldamodel.LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    # Print out the topics
    topics = ldamodel.print_topics( num_words=NUM_WORDS )
    #print("The following topics were generated:")
    #for topic in topics:
      #print( topic )

    # Process the books and get final training data
    X_train = []
    Y_train = genres_train # TODO do I have to preprocess it as well?

    for book in books_train:

      # Get the topic weights
      bow = dictionary.doc2bow( book )
      topics = ldamodel.get_document_topics( bow )

      # Convert the vector of dynamic length to
      # constant length feature vector
      x = [0] * self.NUM_TOPICS
      for topic in topics:
        x[topic[0]] = topic[1]
      X_train.append(x)

    # Prepare our test data in the same way
    X_test = []
    Y_test = genres_test # TODO do I have to preprocess it as well?

    for book in books_test:

      # Get the topic weights
      bow = dictionary.doc2bow( book )
      topics = ldamodel.get_document_topics(bow)

      # Convert the vector of dynamic length to
      # constant length feature vector
      x = [0] * self.NUM_TOPICS
      for topic in topics:
        x[topic[0]] = topic[1]
      X_test.append(x)
    df_train = pd.DataFrame()
    df_test =  pd.DataFrame()
    for i in range(self.NUM_TOPICS):
        column = [entity[i] for entity in X_train]
        column2 = [entity[i] for entity in X_test]
        df_train["topic" + str(i)] = column
        df_test["topic" + str(i)] = column2
    return df_train, df_test